In [2]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [3]:
data2 = np.genfromtxt('data2.dat', delimiter='  ')
data2 = data2.transpose()

y = data2[0]
x = sm.add_constant(data2[1])

In [9]:
#part b - MLE
#http://www.statsmodels.org/0.6.1/examples/notebooks/generated/generic_mle.html
#http://rlhick.people.wm.edu/posts/estimating-custom-mle.html


class part_b(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        t1, t2, sigma = params
        endog = self.endog
        exog = self.exog.squeeze()
        eps = np.log(endog) - t1 - exog**t2
        return - norm(0,sigma).logpdf(eps).sum()
    
    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        self.exog_names.append('constant')
        self.exog_names.append('sigma')
        if start_params == None:
            # Reasonable starting values
            start_params = np.append(np.zeros(self.exog.shape[1]), [.5,.5])
        return super(part_b, self).fit(start_params=start_params, maxiter=maxiter, maxfun=maxfun, **kwds)

    
model_b = part_b(data2[0],data2[1])
result_b = model_b.fit()
print(result_b.summary())

Optimization terminated successfully.
         Current function value: 1.381592
         Iterations: 188
         Function evaluations: 335
                                part_b Results                                
Dep. Variable:                      y   Log-Likelihood:                -138.16
Model:                         part_b   AIC:                             278.3
Method:            Maximum Likelihood   BIC:                             280.9
Date:                Thu, 20 Sep 2018                                         
Time:                        17:55:22                                         
No. Observations:                 100                                         
Df Residuals:                      99                                         
Df Model:                           0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

In [6]:
#part c - GMM
#https://github.com/josef-pkt/misc/blob/master/notebooks/ex_gmm_gamma.ipynb
#https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.GMM.html#statsmodels.sandbox.regression.gmm.GMM
#https://gist.github.com/josef-pkt/6895915


class part_c(GMM):
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        kwds.setdefault('k_moms', 2)
        kwds.setdefault('k_params',2)
        super(part_c, self).__init__(*args, **kwds)

    def momcond(self, params):
        t1,t2 = params   
        #unwrap endog
        endog = self.endog
        exog = self.exog.squeeze()
        eps =  np.log(endog) -t1 - exog**t2
        g = np.column_stack( (eps, eps*exog ))
        return g 

    
model_c = part_c(data2[0],data2[1], None)
beta0 = np.array([1, 1])
model_c.set_param_names(['theta_1','theta_2'])
result_c = model_c.fit(beta0, maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result_c.summary())

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 52
         Function evaluations: 98
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 22
         Function evaluations: 45
                                part_c Results                                
Dep. Variable:                      y   Hansen J:                    1.697e-08
Model:                         part_c   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Thu, 20 Sep 2018                                         
Time:                        15:37:14                                         
No. Observations:                 100                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
theta_1      